# ulab

ulab is a C module for micropython. The aim is to implement a small subset of numpy. We chose those functions and methods that might be useful in the context of a microcontroller. This means low-level data processing. 

# Containers

The basic container, `ndarray` is built on top of the micropython array object, and is declared as

```c
typedef struct _ndarray_obj_t {
	mp_obj_base_t base;
	size_t m, n;
	mp_obj_array_t *data;
	size_t bytes;
} ndarray_obj_t;

```

In turn, the declaration of `mp_obj_array_t` can be found in `objarray.h`, and reads as 

```c
typedef struct _mp_obj_array_t {
    mp_obj_base_t base;
    size_t typecode : 8;
    size_t free : (8 * sizeof(size_t) - 8);
    size_t len; // in elements
    void *items;
} mp_obj_array_t;
```

In order to economise on space and gain on speed while performing computations on data, values in `ndarray` are stored in binary format in `*items`, and not in standard micropython objects. The interpretation of these binary objects is determined by the `typecode`, while the "layout" of the data is given by the `m`, and `n` `size_t` members. 

## Initialising containers

An `ndarray` can be initialised by passing an iterable (linear arrays), or an iterable of iterables (matrices) into the constructor

```python

from ulab import ndarray

a = ndarray([1, 2, 3, 4])
a = ndarray([[1, 2, 3, 4], [2, 3, 4, 5]])
a = ndarray([range(10), range(10)])

```

In addition, the constructor can take a keyword argument, `dtype`, that will force type conversion.

# Standard operations

## mathematical functions

The usual mathematical functions of micropython can be called with either a micropython object, or with ndarrays.

# Special methods


`ndarray` objects support a number of special methods. In particular, if they are of the same shape, two `ndarray`s can be summed, multiplied, divided, subtracted, and raised to a power. These operations work also, if the second operand is an micropython variable. As an example, 

```
from ulab import ndarray

a = 10
b = ndarray([[1, 2, 3], [11, 12, 13]])

c = b + b
d = b + a
```

In addition, bitwise operations work on the underlying byte array (i.e., the `typecode` is not taken into account, the bytearrays are simply `or`ed, `and`ed, etc. If the length of the second operand smaller than that of the first operand, then 